# Built0-in Functions

| ANSI SQL Standard | POstgreSQL Extensions |
--------- | -------
sum  | trunc
max | cbrt
min | pi
cast | random
count | to_num
floor | to_char
ceiling | now


## Business Use Cases

+ Encaptsulate business logic
+ Enforce semantic relationships
+ Validate and transoform data
+ control access to sensitive date
+ Encourage code reuse
+ Don't Repeat YOurself(DRY)




+ Setting up a working environment
+ Creating your first function
+ Data types
+ Polymorphic functions
+ Function overloading
+ Stored procedures
+ Error handling

In [1]:
select CURRENT_TIMESTAMP

(1 row(s) affected)

Total execution time: 00:00:00.002

current_timestamp
2024-09-07 23:42:02.844127+08


In [2]:
select * from customers limit 1

(1 row(s) affected)

Total execution time: 00:00:00.002

customerid,firstname,lastname,address1,address2,city,state,zip,country,region,email,phone,creditcardtype,creditcard,creditcardexpiration,username,password,age,income,gender
1,RNPOJG,FQIVBKWIZC,1287944454 Dell Way,,XCXNWIS,ND,42572,US,1,FQIVBKWIZC@dell.com,1287944454,5,1890860486775636,2012/06,user1,password,49,80000,M


In [4]:
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid limit 10;

(10 row(s) affected)

Total execution time: 00:00:00.002

firstname,lastname,orderid,netamount
RKLQYY,HPUIUYBWZX,10677,5.08
GWPSHI,QWRLBUPCHY,2337,39.06
FCHDNP,FQZNEVVHNA,9077,323.30
HOMUAY,BCTJTZIEZQ,6239,341.44
UZCYBH,NKKNSRDQGC,1187,285.39
YADYNS,VBVFOAENRH,3710,350.87
ZGYPBI,NKKQMREHPQ,379,227.45
ZGYPBI,NKKQMREHPQ,9447,83.31
PFIWAF,MQGNVHRTNH,3075,33.63
PQJVZM,PCCNFVKLZE,5019,256.30


In [5]:
create or replace view cust_orders 
as 
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid limit 10;

CREATE VIEW

Total execution time: 00:00:00.003

In [18]:
select * from cust_orders;

: relation "cust_orders" does not exist

Total execution time: 00:00:00

In [7]:
create function cust_orders()
    returns table (firstname varchar, lastname varchar, orderid int, netamount numeric)
    as $$
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid limit 10
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [8]:
select * from cust_orders();

(10 row(s) affected)

Total execution time: 00:00:00.001

firstname,lastname,orderid,netamount
RKLQYY,HPUIUYBWZX,10677,5.08
GWPSHI,QWRLBUPCHY,2337,39.06
FCHDNP,FQZNEVVHNA,9077,323.30
HOMUAY,BCTJTZIEZQ,6239,341.44
UZCYBH,NKKNSRDQGC,1187,285.39
YADYNS,VBVFOAENRH,3710,350.87
ZGYPBI,NKKQMREHPQ,379,227.45
ZGYPBI,NKKQMREHPQ,9447,83.31
PFIWAF,MQGNVHRTNH,3075,33.63
PQJVZM,PCCNFVKLZE,5019,256.30


In [11]:
create or replace function cust_orders(customerid int)
    returns table (firstname varchar, lastname varchar, orderid int, netamount numeric)
    as $$
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid 
    where c.customerid = cust_orders.customerid
    limit 10
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [17]:
select * from cust_orders(42);

(1 row(s) affected)

Total execution time: 00:00:00.001

firstname,lastname,orderid,netamount
IGAITZ,ADIDJMAYUZ,11465,202.29


In [19]:
drop view if exists cust_orders;
drop function if exists cust_orders();
drop function if exists cust_orders(int);

NOTICE: view "cust_orders" does not exist, skipping

DROP VIEW

DROP FUNCTION

DROP FUNCTION

Total execution time: 00:00:00.002

## Input Parameters and Return Values

### Multiple input parameters


In [21]:
create or replace function my_pow(x double precision, y double precision)
returns double precision
as $$ 
select power(x, y)
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [23]:
select my_pow(1,2)

(1 row(s) affected)

Total execution time: 00:00:00.002

my_pow
1


In [25]:
select my_pow(PI(), log(42))

(1 row(s) affected)

Total execution time: 00:00:00.001

my_pow
6.412068866243689


### Default values

In [27]:
create or replace function my_default(x int = 42) 
RETURNS INT
as $$
    SELECT x;
    $$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [28]:
select my_default()

(1 row(s) affected)

Total execution time: 00:00:00.003

my_default
42


### Using arrays for multiple input values

In [32]:
create or replace function array_sum(int_array int[])
RETURNs int 
as $$
    select sum(el) 
    from unnest(int_array) as arr(el);
    $$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [34]:
select array_sum(array[1,2,3])

(1 row(s) affected)

Total execution time: 00:00:00.001

array_sum
6


## Arrays and multiple return values


In [37]:
create or replace function array_sum_avg(int_array int[])
RETURNS table (array_sum bigint, array_avg numeric)
as $$
select sum(el), avg(el)::numeric(5,2)
from unnest(int_array) as arr(el);
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [39]:
select array_sum_avg(array[1,2,3]) as "Record type";



(1 row(s) affected)

Total execution time: 00:00:00.001

Record type
"(6,2.00)"


In [41]:
select * from array_sum_avg(array[1,2,3])

(1 row(s) affected)

Total execution time: 00:00:00.001

array_sum,array_avg
6,2.00
